In [12]:
from premise import *
import bw2data, bw2io, bw2calc
bw2data.projects.set_current("ecoinvent-3.11-cutoff")

In [2]:
bw2data.databases

Databases dictionary with 41 objects, including:
	HP test
	biochar
	biosphere3
	cdr 1
	ecoinvent-3.11-cutoff
	ei_cutoff_3.11_gcam_SSP2-Base_2025 2025-07-24
	ei_cutoff_3.11_gcam_SSP2-RCP26_2035 2025-07-24
	ei_cutoff_3.11_image_SSP2-M_2050 2025-07-24
	ei_cutoff_3.11_image_SSP2-VLHO_2030_BAU 2025-07-27
	ei_cutoff_3.11_image_SSP2-VLHO_2040 2025-05-26
Use `list(this object)` to get the complete list.

In [7]:
ei_acts = list(set([
    (act["name"], act["reference product"])
    for act in bw2data.Database("ecoinvent-3.11-cutoff")
]))

In [4]:
len(ei_acts)

11465

In [6]:
premise_acts = list(set([
    (act["name"], act["reference product"])
    for act in bw2data.Database("ei_cutoff_3.11_image_SSP2-VLHO_2040 2025-05-26")
]))

In [8]:
new_acts = [
    act for act in premise_acts
    if act not in ei_acts
]

In [9]:
len(new_acts)

2181

In [ ]:
for new in new_acts:
    

In [11]:
import pandas as pd

pd.DataFrame(new_acts, columns=["name", "reference product"]).to_excel("premise_LCIs.xlsx")

In [2]:
clear_inventory_cache()

Inventory cache cleared!


In [14]:
scenarios = [
    {"model": "remind", "pathway": "SSP1-NDC", "year": 2025,},
    {"model": "remind", "pathway": "SSP1-NPi", "year": 2025,},
    {"model": "remind", "pathway": "SSP1-PkBudg650", "year": 2025,},
    {"model": "remind", "pathway": "SSP1-PkBudg1000", "year": 2025,},
    {"model": "remind", "pathway": "SSP2-rollBack", "year": 2025,},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2025,},
    {"model": "remind", "pathway": "SSP2-NPi", "year": 2025,},
    {"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2025,},
    {"model": "remind", "pathway": "SSP2-PkBudg1000", "year": 2025,},
    {"model": "remind", "pathway": "SSP3-rollBack", "year": 2025,},
    {"model": "remind", "pathway": "SSP3-NDC", "year": 2025,},
    {"model": "remind", "pathway": "SSP3-NPi", "year": 2025,},
    {"model": "remind-eu", "pathway": "SSP2-NDC", "year": 2025,},
    {"model": "remind", "pathway": "SSP3-PkBudg1000", "year": 2025,},
    {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2025,},
    {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2050,},
    {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2025,},

    {"model": "image", "pathway": "SSP1-L", "year": 2025,},
    {"model": "image", "pathway": "SSP1-Ma", "year": 2025,},
    {"model": "image", "pathway": "SSP1-VLLO", "year": 2025,},
    {"model": "image", "pathway": "SSP2-L", "year": 2025,},
    {"model": "image", "pathway": "SSP2-M", "year": 2025,},
    {"model": "image", "pathway": "SSP2-VLHO", "year": 2025,},
    {"model": "image", "pathway": "SSP5-H", "year": 2025,},
    {"model": "image", "pathway": "SSP3-H", "year": 2025,},

    {"model": "tiam-ucl", "pathway": "SSP2-Base", "year": 2050,},
    {"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2025,},
    {"model": "tiam-ucl", "pathway": "SSP2-RCP26", "year": 2025,},
    {"model": "tiam-ucl", "pathway": "SSP2-RCP45", "year": 2025,},

    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2050,},
    
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2050,},

    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2050,},
]


ndb = NewDatabase(
        scenarios = scenarios,
        source_db="ecoinvent-3.11-cutoff",
        source_version="3.11",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        biosphere_name="biosphere3"
)


Cannot find the IAM scenario file at /Users/romain/GitHub/premise/premise/data/iam_output_files/remind_SSP3-NPi. Will check online.
Cannot find the IAM scenario file at /Users/romain/GitHub/premise/premise/data/iam_output_files/remind-eu_SSP2-NDC. Will check online.
Cannot find the IAM scenario file at /Users/romain/GitHub/premise/premise/data/iam_output_files/remind_SSP3-PkBudg1000. Will check online.
Cannot find the IAM scenario file at /Users/romain/GitHub/premise/premise/data/iam_output_files/remind-eu_SSP2-NPi. Will check online.
Cannot find the IAM scenario file at /Users/romain/GitHub/premise/premise/data/iam_output_files/remind-eu_SSP2-PkBudg650. Will check online.
Cannot find the IAM scenario file at /Users/romain/GitHub/premise/premise/data/iam_output_files/remind-eu_SSP2-PkBudg1000. Will check online.
Cannot find the IAM scenario file at /Users/romain/GitHub/premise/premise/data/iam_output_files/image_SSP1-L. Will check online.
Cannot find the IAM scenario file at /Users/rom

remind_SSP3-NPi.csv: 11.5MB [00:18, 619kB/s] 


remind_SSP3-NPi.csv downloaded successfully.
Reading remind_SSP3-NPi as CSV file
remind-eu_SSP2-NDC.csv not found locally. Downloading...


remind-eu_SSP2-NDC.csv: 19.4MB [00:30, 626kB/s] 


remind-eu_SSP2-NDC.csv downloaded successfully.
Reading remind-eu_SSP2-NDC as CSV file
remind_SSP3-PkBudg1000.csv not found locally. Downloading...


...

tiam-ucl_SSP2-Base.csv downloaded successfully.
Reading tiam-ucl_SSP2-Base as CSV file
The following variables are missing from the IAM file: /Users/romain/GitHub/premise/premise/data/iam_output_files
+-------------------------------------------------------------------+
|                              Variable                             |
+-------------------------------------------------------------------+
| Final Energy|Transport|Truck|Light|Hydrogen Fuel Cell|Electricity |
|     Final Energy|Transport|Car|Hydrogen Fuel Cell|Electricity     |
+-------------------------------------------------------------------+
Found file: tiam-ucl_SSP2-RCP19
Reading tiam-ucl_SSP2-RCP19 as CSV file
The following variables are missing from the IAM file: /Users/romain/GitHub/premise/premise/data/iam_output_files
+-----------------------------------------------------------------+
|                             Variable                            |
+--------------------------------------------------------

tiam-ucl_SSP2-RCP26.csv: 2.49MB [00:01, 2.28MB/s]


tiam-ucl_SSP2-RCP26.csv downloaded successfully.
Reading tiam-ucl_SSP2-RCP26 as CSV file
The following variables are missing from the IAM file: /Users/romain/GitHub/premise/premise/data/iam_output_files
+-------------------------------------------------------------------+
|                              Variable                             |
+-------------------------------------------------------------------+
| Final Energy|Transport|Truck|Light|Hydrogen Fuel Cell|Electricity |
|     Final Energy|Transport|Car|Hydrogen Fuel Cell|Electricity     |
+-------------------------------------------------------------------+
tiam-ucl_SSP2-RCP45.csv not found locally. Downloading...


tiam-ucl_SSP2-RCP45.csv: 2.51MB [00:01, 1.45MB/s]


tiam-ucl_SSP2-RCP45.csv downloaded successfully.
Reading tiam-ucl_SSP2-RCP45 as CSV file
Done!


In [15]:
ndb.generate_scenario_report()

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.


In [4]:
ndb.update()

Processing scenarios for all sectors: 100%|█| 3/3 [04:59<00:00, 99.98s

Done!



In [5]:
ndb.write_superstructure_db_to_brightway(preserve_original_column=True)

Running all checks...
Minor anomalies found: check the change report.
Running all checks...
Minor anomalies found: check the change report.
Running all checks...
Minor anomalies found: check the change report.
Building superstructure database...
Dropped 20796 duplicate(s).
Scenario difference file exported to /Users/romain/GitHub/premise/dev/export/scenario diff files!
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:35


Title: Writing activities to SQLite3 database:
  Started: 10/11/2025 15:01:55
  Finished: 10/11/2025 15:02:30
  Total time elapsed: 00:00:35
  CPU %: 76.90
  Memory %: 23.07
Created database: super_db_11-10-2025
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.
